In [2]:
import pandas as pd
import os


In [17]:
path = 'Data_inicial/'  # Ruta donde estàn los json

# Lista para almacenar los DataFrames
dataframes = []

# Iteramos sobre los nombres de archivos (del 1 al 13)
for i in range(1, 14):
    file_path = os.path.join(path, f'{i}.json')
    df = pd.read_json(file_path, lines=True)
    dataframes.append(df)

# Generamos un ùnico DF
NewYersey_Full_DF = pd.concat(dataframes, ignore_index=True)



In [18]:
# Seleccionar las primeras 2000 filas
subset_df = NewYersey_Full_DF.head(2000)

# Guardar el DataFrame como un archivo JSON
subset_df.to_json('subset_data.json', orient='records', lines=True)


In [4]:
#print(NewYersey_Full_DF.head(10))  # Muestra las primeras filas del DataFrame combinado

# Muestra un resumen del DataFrame incluyendo el tipo de datos de cada columna
print(NewYersey_Full_DF.info(10))

# Muestra los tipos de datos de todas las columnas
print(NewYersey_Full_DF.dtypes)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1950000 entries, 0 to 1949999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 119.0+ MB
None
user_id    float64
name        object
time         int64
rating       int64
text        object
pics        object
resp        object
gmap_id     object
dtype: object


In [8]:
# Verificar nulos en cada columna y imprimir por separado
for column in NewYersey_Full_DF.columns:
    nulos = NewYersey_Full_DF[column].isnull().sum()
    print(f"Cantidad de nulos en {column}: {nulos}")


Cantidad de nulos en user_id: 0
Cantidad de nulos en name: 0
Cantidad de nulos en time: 0
Cantidad de nulos en rating: 0
Cantidad de nulos en text: 0
Cantidad de nulos en pics: 1894153
Cantidad de nulos en resp: 1728764
Cantidad de nulos en gmap_id: 0


In [7]:
# Reemplazar nulos en la columna 'text' donde 'rating' no es nulo
NewYersey_Full_DF.loc[NewYersey_Full_DF['rating'].notnull(), 'text'] = NewYersey_Full_DF.loc[NewYersey_Full_DF['rating'].notnull(), 'text'].fillna("El cliente no dejó comentarios para su valoración")


In [13]:
print(NewYersey_Full_DF['resp'].dropna().iloc[0])


{'time': 1576097102614, 'text': "Thank you! The team here at Bellibone is thrilled to hear such great feedback 💕We hope you have a rockin' 2020 💫"}


In [14]:

# Suponiendo que NewYersey_Full_DF es tu DataFrame
# Crear un nuevo DataFrame a partir de la columna 'resp'
resp_df = pd.json_normalize(NewYersey_Full_DF['resp'].dropna())

# Concatenar el nuevo DataFrame con el original
NewYersey_Full_DF_prueba = pd.concat([NewYersey_Full_DF.drop(columns=['resp']), resp_df], axis=1)

In [16]:
NewYersey_Full_DF_prueba.columns

Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'gmap_id', 'time',
       'text'],
      dtype='object')

In [5]:
# Contar los valores nulos en la columna 'resp'
nulos_en_resp = NewYersey_Full_DF['resp'].isnull().sum()
print(f"Total de valores nulos en la columna 'resp': {nulos_en_resp}")


Total de valores nulos en la columna 'resp': 1728764


In [6]:
# Excluir las columnas 'resp' y 'pics'
df_sin_resp_pics = NewYersey_Full_DF.drop(columns=['resp', 'pics'])

# Realizar el análisis sin considerar las columnas 'resp' y 'pics'
total_registros = df_sin_resp_pics.shape[0]
print(f"Total de registros: {total_registros}")

# Número de filas con al menos un valor nulo, excluyendo 'resp' y 'pics'
filas_con_nulos = df_sin_resp_pics.isnull().any(axis=1).sum()
print(f"Total de filas con valores nulos (sin 'resp' y 'pics'): {filas_con_nulos}")

# Porcentaje de datos utilizables (sin nulos, excluyendo 'resp' y 'pics')
porcentaje_utilizable = ((total_registros - filas_con_nulos) / total_registros) * 100
print(f"Porcentaje de datos utilizables (sin 'resp' y 'pics'): {porcentaje_utilizable:.2f}%")

# Eliminar filas con valores nulos
df_sin_nulos = df_sin_resp_pics.dropna()

# Contar los duplicados en el DataFrame resultante
duplicados = df_sin_nulos.duplicated().sum()
print(f"Total de filas duplicadas después de eliminar nulos: {duplicados}")

# Eliminar duplicados del DataFrame resultante
df_final = df_sin_nulos.drop_duplicates()

# Calcular el nuevo total de registros después de eliminar duplicados
total_registros_final = df_final.shape[0]
print(f"Total de registros después de eliminar duplicados: {total_registros_final}")

# Calcular el porcentaje final de datos utilizables después de eliminar nulos y duplicados
porcentaje_final = (total_registros_final / total_registros) * 100
print(f"Porcentaje final de datos utilizables después de eliminar nulos y duplicados: {porcentaje_final:.2f}%")



Total de registros: 1950000
Total de filas con valores nulos (sin 'resp' y 'pics'): 864336
Porcentaje de datos utilizables (sin 'resp' y 'pics'): 55.68%
Total de filas duplicadas después de eliminar nulos: 27888
Total de registros después de eliminar duplicados: 1057776
Porcentaje final de datos utilizables después de eliminar nulos y duplicados: 54.24%


In [7]:
# Contar la cantidad de cada calificación en la columna 'rating'
conteo_ratings = df_final['rating'].value_counts().sort_index()

# Calcular el total de registros en df_final
total_registros_final = df_final.shape[0]

# Mostrar los resultados y calcular el porcentaje
for rating, count in conteo_ratings.items():
    porcentaje = (count / total_registros_final) * 100
    print(f"Total de calificaciones {rating}: {count} ({porcentaje:.2f}%)")


Total de calificaciones 1: 76742 (7.26%)
Total de calificaciones 2: 34438 (3.26%)
Total de calificaciones 3: 74687 (7.06%)
Total de calificaciones 4: 181706 (17.18%)
Total de calificaciones 5: 690203 (65.25%)


In [13]:
from collections import Counter
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')

# Unir todas las reseñas en un solo texto
text = ' '.join(df_final['text'].dropna())

# Limpiar el texto (remover puntuación, convertir a minúsculas, etc.)
text = re.sub(r'[^\w\s]', '', text.lower())

# Tokenizar el texto (dividir en palabras)
tokens = word_tokenize(text)

# Remover stopwords (palabras comunes sin valor informativo)
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]

# Contar la frecuencia de las palabras
word_freq = Counter(tokens)

# Obtener las palabras más comunes
palabras_mas_frecuentes = word_freq.most_common(10)  # Puedes ajustar el número de palabras más comunes

print("Palabras más frecuentes:")
for word, freq in palabras_mas_frecuentes:
    print(f"{word}: {freq}")


[nltk_data] Downloading package punkt to /home/acer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/acer/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Palabras más frecuentes:
great: 296723
good: 236131
food: 193370
place: 185428
service: 164117
staff: 112230
nice: 109725
friendly: 97296
best: 88627
always: 84796


In [8]:
from textblob import TextBlob

def analyze_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positivo'
    elif analysis.sentiment.polarity == 0:
        return 'neutro'
    else:
        return 'negativo'

df_final['sentimiento'] = df_final['text'].apply(analyze_sentiment)

# Contar la frecuencia de cada sentimiento
sentiment_counts = df_final['sentimiento'].value_counts()

/tmp/ipykernel_2197/2913478406.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['sentimiento'] = df_final['text'].apply(analyze_sentiment)


In [9]:
# Calcular el total de reseñas
total_sentimientos = sentiment_counts.sum()

# Calcular el porcentaje de cada sentimiento
porcentaje_positivo = (sentiment_counts['positivo'] / total_sentimientos) * 100
porcentaje_negativo = (sentiment_counts['negativo'] / total_sentimientos) * 100
porcentaje_neutro = (sentiment_counts['neutro'] / total_sentimientos) * 100

# Mostrar los resultados
print(f"Porcentaje de reseñas positivas: {porcentaje_positivo:.2f}%")
print(f"Porcentaje de reseñas negativas: {porcentaje_negativo:.2f}%")
print(f"Porcentaje de reseñas neutras: {porcentaje_neutro:.2f}%")

Porcentaje de reseñas positivas: 84.19%
Porcentaje de reseñas negativas: 8.40%
Porcentaje de reseñas neutras: 7.42%


In [ ]:
# Calcular el total de reseñas
total_sentimientos = sentiment_counts.sum()

# Calcular el porcentaje de cada sentimiento
porcentaje_positivo = (sentiment_counts['positivo'] / total_sentimientos) * 100
porcentaje_negativo = (sentiment_counts['negativo'] / total_sentimientos) * 100
porcentaje_neutro = (sentiment_counts['neutro'] / total_sentimientos) * 100

# Mostrar los resultados
print(f"Porcentaje de reseñas positivas: {porcentaje_positivo:.2f}%")
print(f"Porcentaje de reseñas negativas: {porcentaje_negativo:.2f}%")
print(f"Porcentaje de reseñas neutras: {porcentaje_neutro:.2f}%")

Porcentaje de reseñas positivas: 84.19%
Porcentaje de reseñas negativas: 8.40%
Porcentaje de reseñas neutras: 7.42%
